In [18]:
from google.cloud import bigquery
from google.cloud.bigquery import job
import pandas as pd
import os
from tqdm import tqdm
import pandas as pd
PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)
import statsmodels.api as sm 
import numpy as np

In [2]:
# 클라이언트 설정
client = bigquery.Client()
query_name = "1017_쿠폰구매데이터"
# SQL 파일 읽기
# file = './query/쿠팡0403_0509.sql'
file = f'../query/{query_name}.sql'
with open(file, 'r') as file:
    query = file.read()


In [3]:
job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)

In [4]:
df = query_job.to_dataframe()

c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)


In [6]:
df.to_parquet('./0301_1021_earningSpending.parquet')

In [8]:
df.entropy.mean()
df.rename(columns={'9000-12999':'9000-10999'},inplace=True)


0.08151066499372647

In [14]:
df.columns

Index(['User_ID', 'Mean_Spend_Ratio', 'entropy', '0-1000', '1000-2999',
       '3000-4999', '5000-6999', '7000-8999', '9000-12999', '11000 - 12999',
       '13000 - 14999', '15000 - 16999', '17000 - 18999', '19000 - 20999',
       '21000 - 22999', '23000 - 24999', '25000 - 26999', '27000 - 28999',
       '29000 - 30999', '31000 - 32999', '33000 - 34999', '35000 - 36999',
       '37000 - 38999', '39000 - 40999', '41000 -'],
      dtype='object')

- 통계치

In [25]:
def ci_stats(df,col):
    mean = df[col].mean()
    std = df[col].std()
    n = len(df)
    ci_upper = mean + 1.96 * std / np.sqrt(n)
    ci_lower = mean - 1.96 * std / np.sqrt(n)
    
    return round(mean,2), round(ci_upper,2), round(ci_lower,2)

spend_ratio = ci_stats(df,'Mean_Spend_Ratio')
entropy = ci_stats(df,'entropy')

print(spend_ratio, entropy)

(0.68, 0.69, 0.67) (0.08, 0.09, 0.07)


In [32]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
tst = df[['Mean_Spend_Ratio','entropy']]

tst_scaled = sc.fit_transform(tst)
pd.DataFrame(tst_scaled).corr()

,0,1
0,1.000000,0.083172
1,0.083172,1.000000


In [33]:
df.

,User_ID,Mean_Spend_Ratio,entropy,0-1000,1000-2999,3000-4999,5000-6999,7000-8999,9000-12999,11000 - 12999,...,23000 - 24999,25000 - 26999,27000 - 28999,29000 - 30999,31000 - 32999,33000 - 34999,35000 - 36999,37000 - 38999,39000 - 40999,41000 -
0,aacef032-f4c6-4304-8169-f0aa975ece97,0.614805,0.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2ea517ea-32b3-4139-96fc-059aec013ade,0.966455,0.0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,87d794c0-640b-4665-948f-2c33e4aa0cb6,0.770704,0.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,fe770088-5a2d-40ed-a779-221742aa12da,0.223630,0.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,b697738c-ca44-490c-a93e-eba332da8e9b,0.266537,0.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3980,7a7a7574-7487-4642-a4d9-135f536e9ef1,NaN,0.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3981,5d4bef2c-13a5-4f74-8bdb-91bba9267528,NaN,0.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3982,385297ad-9475-4a4a-bf19-04d788599e9f,1.000000,0.0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3983,dfac0da5-73a2-467b-9c0b-79fd3d02e17a,NaN,0.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 소비 강도의 증가에 따른 구매금액 범주 속할 확률 증가분

#### 1만원 이하 구매자의 경우, 탕진 확률이 낮다. 이상의 경우, 탕진 확률이 높다


#### 9월 소량 데이터
- print(np.exp(low_coef)) #1.1만원 이하 -> 강도 증가 시 0.14 배 낮아짐
- print(np.exp(low_mid)) #2.1만원 이하 -> 강도 증가 시 2.15배 높아짐
- print(np.exp(mid)) #3.1만원 이하 -> 강도 증가 시 12배 높아짐
- print(np.exp(high)) #4.1 만 이하, 이상 -> 강도 증가 시 30배 높아짐

In [136]:
def pre_prep(df):
    df['low'] = df[['0-1000', '1000-2999', '3000-4999', '5000-6999','7000-8999', '9000-10999', ]].sum(axis=1)
    df['low-mid'] = df[['11000 - 12999', '13000 - 14999', '15000 - 16999', '17000 - 18999', '19000 - 20999']].sum(axis=1)
    df['mid'] = df[['21000 - 22999', '23000 - 24999', '25000 - 26999', '27000 - 28999','29000 - 30999']].sum(axis=1)
    df['high'] = df[['31000 - 32999', '33000 - 34999', '35000 - 36999', '37000 - 38999', '39000 - 40999', '41000 -']].sum(axis=1)

    df_ = df[['User_ID','Mean_Spend_Ratio','low','low-mid','mid','high']].copy()
    df_ = df_[df_['Mean_Spend_Ratio'].notna()]
    df_ = df_.dropna(subset=['Mean_Spend_Ratio'])  # 결측값 제거
    df_ = df_[~df_['Mean_Spend_Ratio'].isin([float('inf'), -float('inf')])] 

    return df_


In [137]:
def logit_func(df_):
    X = df_['Mean_Spend_Ratio']
    y = df_.iloc[:,2:]
    y['low'] = y['low'].apply(lambda x : 1 if x > 1 else x)
    y['low-mid'] = y['low-mid'].apply(lambda x : 1 if x > 1 else x)
    y['mid'] = y['mid'].apply(lambda x : 1 if x > 1 else x)
    y['high'] = y['high'].apply(lambda x : 1 if x > 1 else x)

    results = {}
    for column in y.columns:
        y_column = y[column]
        X_with_const = sm.add_constant(X)
        model = sm.Logit(y_column, X_with_const).fit()
        results[column] = model.summary()

    return results


In [140]:
import pandas as pd
import numpy as np

def get_results(results):
    extracted_results = {}
    for column, summary in results.items():
        # summary.tables[1]에서 계수와 p-value 추출
        # 첫 번째 행은 상수항이므로, 두 번째 행부터 시작
        for i in range(1, len(summary.tables[1].data)):  # 첫 번째 행(상수항)을 제외하고 반복
            coef = summary.tables[1].data[i][1]  # 계수
            p_value = summary.tables[1].data[i][4]  # p-value
            variable_name = summary.tables[1].data[i][0]  # 변수 이름
            extracted_results[f"{column}_{variable_name}"] = {'coef': coef, 'p-value': p_value}        

    # DataFrame 생성
    df = pd.DataFrame(extracted_results).T

    # odds-ratio 계산
    def safe_exp(x):
        try:
            # x를 float로 변환
            float_x = float(x)
            return np.exp(float_x)  # exp 적용
        except (ValueError, TypeError):
            return np.nan  # 변환 불가능한 경우 NaN 반환

    df['odds-ratio'] = df['coef'].apply(safe_exp)
        
    return df

In [141]:
df_result = get_results(logit_func(pre_prep(df)))
df_result

Optimization terminated successfully.
         Current function value: 0.484430
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.342526
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.141695
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.269407
         Iterations 8


,coef,p-value,odds-ratio
low_const,2.7560,0.000,15.736770
low_Mean_Spend_Ratio,-1.9041,0.000,0.148957
low-mid_const,-2.6447,0.000,0.071027
low-mid_Mean_Spend_Ratio,0.7709,0.000,2.161711
mid_const,-5.2627,0.000,0.005181
mid_Mean_Spend_Ratio,2.4887,0.000,12.045607
high_const,-5.0262,0.000,0.006564
high_Mean_Spend_Ratio,3.3980,0.000,29.904232
